In [574]:
import pandas as pd
import numpy as np
from string import punctuation
import random
import math
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import nltk
from nltk.corpus import stopwords
from nltk.corpus import movie_reviews
from nltk import word_tokenize
from nltk.util import ngrams
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from spellchecker import SpellChecker
from datetime import datetime, timedelta
import re,string,math

In [575]:
def change_dual_words(word):
    first=''
    second=''
    def get_correct_word(w):
        if w=="sha":
            return("shall")
        elif w=="wo":
            return("will")
        elif w=="y":
            return("you")
        else:
            return(w)
#     word=word.replace('$','$').replace('+','')
    word=word.strip().strip(':').strip("'").strip(',').strip('`').strip('[').strip(']').strip('-').strip('_').strip('#').lower()
    word=word.strip('?').strip('+').strip('/').strip('(').strip(')').strip(';').strip('!').strip('.').strip('?')
    if word.endswith("n't"):
        first=get_correct_word(word[:word.index("n't")])
        second="not"
    elif word.endswith("'ll"):
        first=get_correct_word(word[:word.index("'ll")])
        second="will"
    elif word.endswith("'ve"):
        first=get_correct_word(word[:word.index("'ve")])
        second="have"
    elif word.endswith("'re"):
        first=get_correct_word(word[:word.index("'re")])
        second="are"
    elif word.endswith("'d"):
        first=get_correct_word(word[:word.index("'d")])
        second="did"
    elif word.endswith("'s"):
        first=get_correct_word(word[:word.index("'s")])
        second="is"
    elif word.endswith("'all"):
        first=get_correct_word(word[:word.index("'all")])
        second="all"
    else:
        first=word
        second="<allclear>"
    return((first,second))

In [616]:
def cleanLine(line):
    line=line.encode('ascii','ignore').decode(encoding = "ISO-8859-1")
    line=line.replace('&#36;','DOLLAR').replace('+',' ').replace('$','DOLLAR')
    
    tokenizer = RegexpTokenizer('\w+-+\w+')
    hyphenated_word=tokenizer.tokenize(line)
    for word in hyphenated_word:
        line=line.replace(word,word.replace('-',' '))
        
    tokenizer = RegexpTokenizer('\w+,+\w+')
    hyphenated_word=tokenizer.tokenize(line)
    for word in hyphenated_word:
        line=line.replace(word,word.replace(',',' '))
        
        
    tokenizer = RegexpTokenizer('\D+/+\D+')
    hyphenated_word=tokenizer.tokenize(line)
    for word in hyphenated_word:
        line=line.replace(word,word.replace('/',' '))
        
    table = str.maketrans(dict.fromkeys(string.punctuation))
    line=line.translate(table)
    
#     print(hyphenated_word)

    word_lst=line.split()
    clean_lst=[]
    for idx in range(0,len(word_lst)):
        if word_lst[idx] not in punctuation:
            dual_words=change_dual_words(word_lst[idx])
            if dual_words[1]=='<allclear>':
                    clean_lst.append(dual_words[0])
            else:
                    clean_lst.append(dual_words[0])
                    clean_lst.append(dual_words[1])
#     processed_lst.append('</s>')
    return(str(' ').join(clean_lst))

In [617]:
def get_relevance_date(dt):
    if dt.weekday()==5:
        return(dt - timedelta(1))
    elif dt.weekday()==6:
        return(dt - timedelta(2))
    else:
        return(dt)

In [618]:
to_datetime = lambda d: datetime.strptime(d, "%d/%m/%Y")
ft_data=pd.read_csv('../Data/NewsData/financial_times.csv',sep='|',names=['Date', 'HeadLines'],converters={'Date': to_datetime},encoding = "ISO-8859-1")
mc_data=pd.read_csv('../Data/NewsData/moneycontrol.csv',sep='|',names=['Date', 'HeadLines'],converters={'Date': to_datetime},encoding = "ISO-8859-1")
op_data=pd.read_csv('../Data/NewsData/oilprice.csv',sep='|',names=['Date', 'HeadLines'],converters={'Date': to_datetime},encoding = "ISO-8859-1")
mc_data.drop(index=mc_data.iloc[mc_data.HeadLines.isin(['Commodities@moneycontrol \\','Commodities@Moneycontrol \\','Editor\'s Take \\','Podcast \\']).values].index,axis=0,inplace=True)
data = pd.concat([ft_data,mc_data,op_data],axis=0,sort=False)

In [619]:
mc_data[mc_data.HeadLines.str.contains('\$')].HeadLines.str.replace('$','DOLLAR')

5       Oil slips to DOLLAR62 as trade talks between U...
83      Oil slips further below DOLLAR58 as economic g...
107     Oil falls below DOLLAR64 on prospects for Saud...
130     Drone attack on Saudi plants may lift oil pric...
148     Oil rises to DOLLAR61 on trade hopes; US inven...
                              ...                        
1751    Oil slips towards DOLLAR50 on doubts over outp...
1763    Oil could collapse into the DOLLAR30s if OPEC ...
1793    Oil prices could jump DOLLAR10 a barrel if OPE...
6267    Brent crude drops below DOLLAR111 on renewed g...
6851    Oil slips to DOLLAR123 after China cuts growth...
Name: HeadLines, Length: 85, dtype: object

In [620]:
data['Lemmetized_HL']=data.HeadLines.apply(lambda line:cleanLine(line)).str.split().apply(lambda word_lst: ' '+str(' ').join([wnl.lemmatize(word) for word in word_lst])+' ')

In [621]:
data['Agg_Col']=data.Date.apply(lambda dt:datetime.strftime(get_relevance_date(dt),"%d/%m/%Y"))

In [622]:
def upd_word_doc_count(data,full_refresh):
    try:
        if not full_refresh:
            word_lst=data.str.split(expand=True).stack().unique()
            word_doc_cnt_df=pd.read_csv('../Data/Processed_Data/word_doc_count.csv',sep='|',index_col=0)
            for word in word_lst:
                if word in word_doc_cnt_df.index:
                    word_doc_cnt_df.DocCount[word]=word_doc_cnt_df.DocCount[word]+data.str.contains(' '+word.replace('$','\\$')+' ').sum()
                else:
                    word_doc_cnt_df.DocCount[word]=data.str.contains(' '+word.replace('$','\\$')+' ').sum()
            word_doc_cnt_df.to_csv('../Data/Processed_Data/word_doc_count.csv',sep='|')
        else:
            word_lst=data.str.split(expand=True).stack().unique()
            wc={word:data.str.contains(' '+word+' ').sum() for word in word_lst}
            pd.DataFrame.from_dict(wc,orient='index',columns=['DocCount']).to_csv('../Data/Processed_Data/word_doc_count.csv',sep='|')
    except (pd.errors.EmptyDataError,FileNotFoundError):
        exc_type, exc_value, exc_traceback = sys.exc_info()
        print('Error found at')
        traceback.print_tb(exc_traceback, limit=1, file=sys.stdout)

In [623]:
upd_word_doc_count(data.Lemmetized_HL,True)
word_doc_cnt_df=pd.read_csv('../Data/Processed_Data/word_doc_count.csv',sep='|',index_col=0)
word_doc_cnt_df['IDF']=word_doc_cnt_df.DocCount.apply(lambda dc: math.log(data.shape[0]/dc))
word_doc_cnt_df.to_csv('../Data/Processed_Data/word_doc_count.csv',sep='|')

In [637]:
def get_tf_idf(line,wrdidf):
#     word_doc_cnt_df=pd.read_csv('../Data/Processed_Data/word_doc_count.csv',sep='|',index_col=0)
    tf_idf_score=0
    word_lst=line.split()
    for word in word_lst:
        tf_idf_score+=(line.count(word)/len(word_lst))*wrdidf[word]
    return(tf_idf_score)

In [638]:
data['TFIDF_SCORE']=data.Lemmetized_HL.apply(lambda line: get_tf_idf(line,word_doc_cnt_df['IDF']))

In [639]:
tfidf_score=data.groupby(['Agg_Col']).sum()

In [641]:
tfidf_score.to_csv('../Data/Processed_Data/tfidf_score.csv',sep='|')